In [1]:

# %matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

import pandas as pd
import numpy as np

import random
np.random.seed(1234)
random.seed(1234)

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


train_df = pd.read_csv("/content/drive/MyDrive/train.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test.csv")
submission = pd.read_csv("/content/drive/MyDrive/sample_submission.csv")

all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True) #목적변수

#데이터 처리

#데이터를 train_df와 test_df로 되돌리기

train_df_le = all_df[~all_df["SalePrice"].isnull()] #Sales price가 결측치가 아닌 행으로 이루어진 데이터 프레임
test_df_le = all_df[all_df["SalePrice"].isnull()] #Sales price가 결측치가 아닌 행으로 이루어진 데이터 프레임

train_X = train_df_le.drop(["SalePrice", "Id"], axis=1)
train_Y = train_df_le["SalePrice"]

#데이터 전처리

pd.options.mode.chained_assignment = None
train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])
train_X = train_df_le.drop(["SalePrice","SalePrice_log","Id"], axis=1)
train_Y = train_df_le["SalePrice_log"]

all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)
categories = all_df.columns[all_df.dtypes == "object"]

all_df = pd.concat([train_df, test_df], sort=False).reset_index(drop=True)
categories = all_df.columns[all_df.dtypes == "object"]
print(categories)

all_df.isnull().sum().sort_values(ascending=False).head(40)#결측치가 많은 변수 알아보기

#결측치가 많은 고급 주택 설비에 관한 변수 정리하기

#PoolGC
all_df.PoolQC.value_counts()
all_df.loc[~all_df["PoolQC"].isnull(), "PoolQC"] = 1
all_df.loc[all_df["PoolQC"].isnull(), "PoolQC"] = 0

#MisFeature
all_df.loc[~all_df["MiscFeature"].isnull(), "MiscFeature"] = 1
all_df.loc[all_df["MiscFeature"].isnull(), "MiscFeature"] = 0

#Alley
all_df.loc[~all_df["Alley"].isnull(), "Alley"] = 1
all_df.loc[all_df["Alley"].isnull(), "Alley"] = 0

#HighFacility_col 정의

HighFacility_col = ["PoolQC","MiscFeature","Alley"]
for col in HighFacility_col:
    if all_df[col].dtype == "object":
        if len(all_df[all_df[col].isnull()]) > 0:
            all_df.loc[~all_df[col].isnull(), col] = 1
            all_df.loc[all_df[col].isnull(), col] = 0

all_df["hasHighFacility"] = all_df["PoolQC"] + all_df["MiscFeature"] + all_df["Alley"]
all_df["hasHighFacility"] = all_df["hasHighFacility"].astype(int)

#PoolGC, MisFeature, Alley 삭제하기
all_df = all_df.drop(["PoolQC","MiscFeature","Alley"],axis=1)

"""이상치 제거하기"""

#수치 데이터만 추출
train_df_num = train_df.select_dtypes(include=[np.number])
nonratio_features = ["Id", "MSSubClass", "OverallQual", "OverallCond", "YearBuilt", "YearRemodAdd", "MoSold", "YrSold"]
num_features = sorted(list(set(train_df_num) - set(nonratio_features)))

train_df_num_rs = train_df_num[num_features]

all_df = all_df[(all_df['BsmtFinSF1'] < 2000) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['TotalBsmtSF'] < 3000) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['GrLivArea'] < 4500) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['1stFlrSF'] < 2500) | (all_df['SalePrice'].isnull())]
all_df = all_df[(all_df['LotArea'] < 100000) | (all_df['SalePrice'].isnull())]

categories = categories.drop(["PoolQC","MiscFeature","Alley"])

for cat in categories:
    le = LabelEncoder()

    all_df[cat].fillna("missing", inplace=True)
    le = le.fit(all_df[cat])
    all_df[cat] = le.transform(all_df[cat])
    all_df[cat] = all_df[cat].astype("category")

train_df_le = all_df[~all_df["SalePrice"].isnull()]
test_df_le = all_df[all_df["SalePrice"].isnull()]

train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])
train_X = train_df_le.drop(["SalePrice","SalePrice_log", "Id"], axis=1)
train_Y = train_df_le["SalePrice_log"]

"""특징량 생성"""

all_df["Age"] = all_df["YrSold"] - all_df["YearBuilt"]
train_df_le = all_df[~all_df["SalePrice"].isnull()]
test_df_le = all_df[all_df["SalePrice"].isnull()]

train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])
train_X = train_df_le.drop(["SalePrice","SalePrice_log","Id"], axis=1)
train_Y = train_df_le["SalePrice_log"]

"""넓이 변수로 새로운 특징량 작성"""

all_df["TotalSF"] = all_df["TotalBsmtSF"] + all_df["1stFlrSF"] + all_df["2ndFlrSF"]
all_df["Total_Bathrooms"] = all_df["FullBath"] + all_df["HalfBath"] + all_df["BsmtFullBath"] + all_df["BsmtHalfBath"]
all_df["Total_PorchSF"] = all_df["WoodDeckSF"] + all_df["OpenPorchSF"] + all_df["EnclosedPorch"] + all_df["3SsnPorch"] + all_df["ScreenPorch"]
all_df["hasPorch"] = all_df["Total_PorchSF"].apply(lambda x: 1 if x > 0 else 0)
all_df = all_df.drop("Total_PorchSF",axis=1)

train_df_le = all_df[~all_df["SalePrice"].isnull()]
test_df_le = all_df[all_df["SalePrice"].isnull()]

train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])
train_X = train_df_le.drop(["SalePrice","SalePrice_log","Id"], axis=1)
train_Y = train_df_le["SalePrice_log"]

test_X = test_df_le.drop(["SalePrice", "Id"], axis=1)




Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')


In [5]:
# 랜덤포레스트 학습

from sklearn.ensemble import RandomForestRegressor as rf

hasnan_cat = []
for col in all_df.columns:
   tmp_null_count = all_df[col].isnull().sum()
   if(tmp_null_count>0)&(col!="SalePrice"):
     print(col, tmp_null_count)
     hasnan_cat.append(col)

for col in all_df.columns:
  tmp_null_count = all_df[col].isnull().sum()
  if(tmp_null_count>0) & (col != "SalePrice"):
     print(col, tmp_null_count)
     all_df[col] = all_df[col].fillna(all_df[col].median())

train_df_le = all_df[~all_df["SalePrice"].isnull()]
test_df_le = all_df[all_df["SalePrice"].isnull()]
train_df_le["SalePrice_log"] = np.log(train_df_le["SalePrice"])

train_X = train_df_le.drop(["SalePrice", "SalePrice_log", "Id"], axis=1)
train_Y = train_df_le["SalePrice_log"]


In [9]:

from sklearn.model_selection import KFold

folds = 3
kf = KFold(n_splits=folds)

models_rf= []
rmses_rf = []
oof_rf = np.zeros(len(train_X))

for train_index, val_index in kf.split(train_X):
  X_train = train_X.iloc[train_index]
  X_valid = train_X.iloc[val_index]
  y_train = train_Y.iloc[train_index]
  y_valid = train_Y.iloc[val_index]

  model_rf=rf( n_estimators=50, random_state=1234)

  model_rf.fit(X_train, y_train)
  y_pred = model_rf.predict(X_valid)
  tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
  print(tmp_rmse)
  models_rf.append(model_rf)
  rmses_rf.append(tmp_rmse)
  oof_rf[val_index]=y_pred

sum(rmses_rf)/len(rmses_rf)


0.13832809756301317
0.1419408245515443
0.1295288287348652


0.13659925028314088

In [10]:
# Opentuna 최적화

!pip install optuna

import optuna
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_Y, test_size=0.2, random_state=1234, shuffle=False,  stratify=None)

def objective(trial):

    # 하이퍼파라미터 추천 범위 지정
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "min_samples_split": trial.suggest_float("min_samples_split", 0.1, 1.0),
        "min_samples_leaf": trial.suggest_float("min_samples_leaf", 0.1, 0.5),
        "max_features": trial.suggest_categorical("max_features", ["auto", "sqrt", "log2"]),
        "random_state": 1234
    }

    # 모델 훈련
    model_rf = RandomForestRegressor(**params)
    model_rf.fit(X_train, y_train)

    # 검증 데이터에 대한 예측
    y_pred = model_rf.predict(X_valid)

    # RMSE 계산
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    return rmse

# Optuna 최적화 실행
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# 최적 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print("Value: ", trial.value)
print("Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")



[I 2023-12-03 09:14:50,907] A new study created in memory with name: no-name-e1008b3a-19c0-4d89-948b-8491135bda73
[I 2023-12-03 09:14:51,038] Trial 0 finished with value: 0.3785787794296114 and parameters: {'n_estimators': 71, 'max_depth': 13, 'min_samples_split': 0.7641268170480775, 'min_samples_leaf': 0.2317247104399364, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.3785787794296114.
[I 2023-12-03 09:14:51,283] Trial 1 finished with value: 0.3785599110642524 and parameters: {'n_estimators': 172, 'max_depth': 13, 'min_samples_split': 0.8396046616036869, 'min_samples_leaf': 0.36433059590610506, 'max_features': 'sqrt'}. Best is trial 1 with value: 0.3785599110642524.
[I 2023-12-03 09:14:51,416] Trial 2 finished with value: 0.3785584317408332 and parameters: {'n_estimators': 88, 'max_depth': 8, 'min_samples_split': 0.5834780740343772, 'min_samples_leaf': 0.3475866001156307, 'max_features': 'log2'}. Best is trial 2 with value: 0.3785584317408332.
[I 2023-12-03 09:14:51,557] Trial

Best trial:
Value:  0.20051916928894706
Params: 
    n_estimators: 127
    max_depth: 6
    min_samples_split: 0.12454381450732104
    min_samples_leaf: 0.10057970244240688
    max_features: auto


In [13]:
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Optuna를 사용하여 최적의 하이퍼파라미터를 찾은 결과
best_params = study.best_params

# KFold 설정
folds = 3
kf = KFold(n_splits=folds)

# 모델 초기화
models_rf = []
rmses_rf = []
oof_rf = np.zeros(len(train_X))
r2_scores_rf = []
training_times_rf = []

# KFold에 따라 모델 학습 및 예측 수행
for train_index, val_index in kf.split(train_X):
    X_train = train_X.iloc[train_index]
    X_valid = train_X.iloc[val_index]
    y_train = train_Y.iloc[train_index]
    y_valid = train_Y.iloc[val_index]

    # 최적의 하이퍼파라미터로 랜덤 포레스트 모델 생성
    model_rf = RandomForestRegressor(**best_params)

    # 모델 학습 시간 측정 시작
    start_time = time.time()

    # 모델 학습
    model_rf.fit(X_train, y_train)

    # 모델 학습 시간 측정 종료
    end_time = time.time()
    training_time = end_time - start_time
    training_times_rf.append(training_time)  # 학습 시간 저장

    # 검증 데이터에 대한 예측
    y_pred = model_rf.predict(X_valid)

    # RMSE 계산 및 출력
    tmp_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    print("RMSE:", tmp_rmse)

    # 결과 저장
    models_rf.append(model_rf)
    rmses_rf.append(tmp_rmse)
    oof_rf[val_index] = y_pred

   # R^2 계산 및 출력
    r2 = r2_score(y_valid, y_pred)
    print("R^2:", r2)

   # 결과 저장
    models_rf.append(model_rf)
    rmses_rf.append(tmp_rmse)
    r2_scores_rf.append(r2)
    oof_rf[val_index] = y_pred

# 교차 검증 결과에 대한 평균 RMSE 출력
avg_rmse_rf = sum(rmses_rf) / len(rmses_rf)
print("Average RMSE across folds:", avg_rmse_rf)

# 교차 검증 결과에 대한 평균 R^2 출력
avg_r2_rf = sum(r2_scores_rf) / len(r2_scores_rf)
print("Average R^2 across folds:", avg_r2_rf)

# 전체 모델의 평균 학습 시간 출력
avg_training_time_rf = sum(training_times_rf) / len(training_times_rf)
print("Average training time across folds:", avg_training_time_rf)

RMSE: 0.20275811225505772
R^2: 0.7402144159051612
RMSE: 0.21731344719646764
R^2: 0.7101074986638465
RMSE: 0.19725321026812548
R^2: 0.7256430233576585
Average RMSE across folds: 0.20577492323988364
Average R^2 across folds: 0.7253216459755555
Average training time across folds: 1.392430067062378
